In [1]:
# import statements
from time import sleep
from json import dumps
from kafka import KafkaProducer
import random                                 #import libraries
import datetime as dt
import csv




In [2]:
def read_csv(fileName):
    data  = []
    input_file = csv.DictReader(open(fileName))
    for each in input_file:                               # create function to read data
        data.append(each)
    return data

def get_random_no(a,b,all_to_send,i):
    no_of_rows = random.randint(a,b)
    each_machine_send= all_to_send[i][:no_of_rows]
    rem = no_of_rows - len(each_machine_send)            # get random no of rows for "X" no of records and "Y" no of records
 
    return (no_of_rows,rem,each_machine_send)      #return  rem rows to keep an eye for exhaustion of data


                                                    
def machine_data(n,cRows,machine_list):
    chosen_data = []                                       # segregate dat based on machine id
                                                                                    
    for each in cRows:
        if each["machine"] == machine_list[n]:
            chosen_data.append(each)
    return chosen_data
    
                      #WRITE THE CODE TO READ CSV FILE
    
def publish_message(producer_instance, topic_name, data):
    try:
        producer_instance.send(topic_name, data)
        #print('Message published successfully. Data: ' + str(data))
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))
        
def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                                  value_serializer=lambda x: dumps(x).encode('ascii'),
                                  api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka.')
        print(str(ex))
    finally:
        return _producer
    
if __name__ == '__main__':
   
    topic = 'memory'        #  give topic name
    cRows = read_csv('Streaming_Linux_memory.csv')           # read data from function above
    machine_list = []                  # initialize empty list
    
    for i in cRows:
        i['sequence'] = int(i['sequence'])
        i['machine'] = int(i['machine'])
        i['PID'] = int(i['PID'])
        i['MINFLT'] = int(i['MINFLT'])
        i['MAJFLT'] = int(i['MAJFLT'])        # convert to appropraite data type
        i['VSTEXT'] = int(i['VSTEXT'])
        i['VSIZE'] = float(i['VSIZE'])
        i['RSIZE'] = float(i['RSIZE'])
        i['VGROW'] = float(i['VGROW'])
        i['RGROW'] = float(i['RGROW'])
        i['MEM'] = float(i['MEM'])


        if i['machine'] in machine_list:         # find no of machines and sort list
            continue
        else:
            machine_list.append(i['machine'])
    machine_list.sort() 
    
           

    
    print('Publishing records..')
    producer = connect_kafka_producer()
    all_to_send = []
    n=0
    length_chosen_data_list = []
    temp = []
    while n <= (len(machine_list)-1):                                    # filter and store data as per machine id in seperate lists
        chosen_data = machine_data(n,cRows,machine_list)
        all_to_send.append(chosen_data)
        temp.append(chosen_data)
        length_chosen_data_list.append(len(chosen_data))
        n=n+1
    init = 0
    temp
    while True:
        temp_y= []
        for i in range(len(all_to_send)):               # fr every machine send records
            x_send = []
            y_send = []
            # continous streaming 
            start_index = 0
            no_of_rows_x,rem_x,each_machine_send_x = get_random_no(20,80,all_to_send,i)
            #print(each_machine_send_x)                                                      #get random no of rows 
            #print(len(each_machine_send_x))
            #print(len(all_to_send[i]))
            all_to_send[i] = list(filter(lambda x: x not in each_machine_send_x,all_to_send[i]))  # filter data so that it does not repeat
            #print(len(all_to_send[i]))

            no_of_rows_y,rem_y,each_machine_send_y = get_random_no(0,5,all_to_send,i)
            #print(each_machine_send_y)
            all_to_send[i] = list(filter(lambda x: x not in each_machine_send_y,all_to_send[i]))
 
            if len(all_to_send[i]) == 0:
                all_to_send[i] = temp[i]
            if rem_x > 0:
                all_to_send[i] = temp[i]                            # conditions for exhaustion of data by replaceing original values in order to resratrt from first sequence
                each_machine_send_x = all_to_send[i][:rem_x]
            if rem_y > 0:
                all_to_send[i] = temp[i]
                each_machine_send_y = all_to_send[i][:rem_y]
            count = 0
            for j in range(len(each_machine_send_y)):
                ts = {'ts': int(dt.datetime.now().timestamp())}          # generate  y and x with same time stamp
                data_x=dict(each_machine_send_x[j],**ts)                  # and stre y no of records  in a temporary list
                x_send.append(data_x)
                data_y=dict(each_machine_send_y[j],**ts)
                y_send.append(data_y)
                count = count + 1 
            temp_y.append(y_send)
            for k in range(count,len(each_machine_send_x)):
                ts = {'ts': int(dt.datetime.now().timestamp())}
                temp_data_x = dict(each_machine_send_x[k],**ts)        # append time stamp to the remaining records 
                x_send.append(temp_data_x) 
            #print(x_send)
            for each in x_send:
                print(each)                                          # send "X" records
                publish_message(producer, topic, each)
                
        sleep(10)                                    # after every cycle of previous x records  publish y records from temporary variable
        for indv in range(len(temp_y)):                            
            for ech in temp_y[indv]:        
                print("hello" , ech )
                publish_message(producer, topic, ech)
    


Publishing records..
{'sequence': 1, 'machine': 4, 'PID': 4392, 'MINFLT': 221, 'MAJFLT': 0, 'VSTEXT': 596, 'VSIZE': 184.3, 'RSIZE': 5668.0, 'VGROW': 184.3, 'RGROW': 5668.0, 'MEM': 0.0, 'CMD': 'apache2', 'ts': 1616378573}
{'sequence': 2, 'machine': 4, 'PID': 4397, 'MINFLT': 221, 'MAJFLT': 0, 'VSTEXT': 596, 'VSIZE': 184.3, 'RSIZE': 5668.0, 'VGROW': 184.3, 'RGROW': 5668.0, 'MEM': 0.0, 'CMD': 'apache2', 'ts': 1616378573}
{'sequence': 3, 'machine': 4, 'PID': 4398, 'MINFLT': 221, 'MAJFLT': 0, 'VSTEXT': 596, 'VSIZE': 184.3, 'RSIZE': 5668.0, 'VGROW': 184.3, 'RGROW': 5668.0, 'MEM': 0.0, 'CMD': 'apache2', 'ts': 1616378573}
{'sequence': 4, 'machine': 4, 'PID': 4400, 'MINFLT': 221, 'MAJFLT': 0, 'VSTEXT': 596, 'VSIZE': 184.3, 'RSIZE': 5668.0, 'VGROW': 184.3, 'RGROW': 5668.0, 'MEM': 0.0, 'CMD': 'apache2', 'ts': 1616378573}
{'sequence': 5, 'machine': 4, 'PID': 4401, 'MINFLT': 221, 'MAJFLT': 0, 'VSTEXT': 596, 'VSIZE': 184.3, 'RSIZE': 5668.0, 'VGROW': 184.3, 'RGROW': 5668.0, 'MEM': 0.0, 'CMD': 'apache2

KeyboardInterrupt: 